In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import itertools
import re

plt.rcParams['figure.figsize'] = (16, 6)
#plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams['figure.constrained_layout.use'] = True # Experimental

sns.set_style('darkgrid')

In [ ]:
def save_fig(fig, name, parent_dir='../figures/paper/', exts=['png','pdf']):
    
    p = Path(parent_dir)
    p.mkdir(exist_ok=True)
    
    for ext in exts:
        fn = p/f'{name}.{ext}'
        fig.savefig(fn)
    
    fig.show()

In [ ]:
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")#, index_col=[0,1,2,3,4,5,6])
df = df.set_index([c for c in df.columns if c != 'value'], verify_integrity=True)

esc_markers = {
    'hvdc':'X',
    'pipeline-h2':'s', 
    'pipeline-ch4':'p', 
    'shipping-lh2':'^', 
    'shipping-lohc':'d', 
    'shipping-lch4':'v', 
    'shipping-lnh3':'<', 
    'shipping-meoh':'>',
}

cmap = matplotlib.cm.get_cmap('tab10')
exp_colors = {
    'AU':cmap(0),
    'AR':cmap(1), 
    'ES':cmap(2), 
    'EG':cmap(3), 
    'MA':cmap(4), 
    'SA':cmap(5), 
    'DK':cmap(6), 
    'DE':cmap(7),
}

df = df.reorder_levels(['year','wacc','importer','category','subcategory','exporter','esc'])

plt.figure(figsize=(10,10))
ax = plt.gca()

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['general','Cost per MWh delivered'].loc[name]['value']
    y = group.loc['general','Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('EUR per MWh delivered')

# Construct legend
legend_esc = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='ESC', marker=None)]
legend_esc += [matplotlib.lines.Line2D([], [], color='white', linestyle='None', markersize=10, markeredgecolor='black',
                                      label=n, marker=m) for n,m in esc_markers.items()]

legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='.',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_esc+legend_exp, ncol=2, loc='lower right')

# save figure
save_fig(plt.gcf(), "esf_vs_LCoEnergy")

In [ ]:
plt.figure(figsize=(10,10))
ax = plt.gca()

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['RES', 'Curtailed electricity'].loc[name]['value']/group.loc['RES', 'Total produced electricity'].loc[name]['value']*100.
    y = group.loc['general', 'Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('Curtailed electricity [%]')

# Construct legend
legend_esc = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='ESC', marker=None)]
legend_esc += [matplotlib.lines.Line2D([], [], color='white', linestyle='None', markersize=10, markeredgecolor='black',
                                      label=n, marker=m) for n,m in esc_markers.items()]

legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='.',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_esc+legend_exp, ncol=2, loc='lower right')

# save figure
save_fig(plt.gcf(), "esf_vs_curtailment")

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,6))

ax = ax1

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['general','Cost per MWh delivered'].loc[name]['value']
    y = group.loc['general','Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('EUR per MWh delivered')

# Construct legend
legend_esc = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='ESC', marker=None)]
legend_esc += [matplotlib.lines.Line2D([], [], color='white', linestyle='None', markersize=10, markeredgecolor='black',
                                      label=n, marker=m) for n,m in esc_markers.items()]

legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='.',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_esc+legend_exp, ncol=2, loc='lower right')

ax = ax2

tdf = df.loc[2030,'homogeneous','DE']

for name, group in tdf.groupby(['exporter','esc']):
    x = group.loc['RES', 'Curtailed electricity'].loc[name]['value']/group.loc['RES', 'Total produced electricity'].loc[name]['value']*100.
    y = group.loc['general', 'Energy surplus factor'].loc[name]['value']

    ax.scatter(x,y,
               marker=esc_markers[name[1]],
               color=exp_colors[name[0]],
               s=50)
    
#ax.set_ylabel('Energy surplus factor')
ax.set_xlabel('Curtailed electricity [%]')

# save figure
save_fig(plt.gcf(), "esf_vs_LCoEnergy-and-curtailment")

In [ ]:
## Supply curves (electricity)
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/2030_homogeneous/hvdc/{exp}-DE/lcoes.csv"

from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

fig = plt.figure(figsize=(8,6))
ax = plt.gca()

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax.plot(df['cumulative generation']/1e6, #in TWh
            df['lcoe'], '-X',
            ms=5,
            color=exp_c, label=exp)

#ax.legend(ncol=len(exp_colors), loc='lower center')
ax.legend()
ax.set_xlim(1,2.5e3)
ax.set_ylim(0,125)
ax.set_ylabel('Marginal cost of electricity [EUR/MWh]')
ax.set_xlabel('Annual electricity potential [TWh]')

## Create a set of inset Axes: these should fill the bounding box allocated to
# them.
ax2 = plt.axes([0,0,1,1], frameon=True)
# Manually set the position and relative size of the inset axes within ax1
ip = InsetPosition(ax1, [0.3,0.65,0.6,0.3])
ax2.set_axes_locator(ip)
ax.add_patch(plt.Rectangle((0.04, .565), .415, .415, ls="-", ec="c", fc="white",
                           transform=ax.transAxes))

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax2.plot(df['cumulative generation']/1e6, #in TWh
            df['lcoe'], '-',
            ms=5,
            color=exp_c, label=exp)

ax2.set_ylim(0,200)
ax2.set_xlabel("[TWh]")
ax2.set_ylabel("[EUR/MWh]")

# save figure
save_fig(plt.gcf(), "supply-curves")

In [ ]:
## Share by technology of electricity
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/2030_homogeneous/hvdc/{exp}-DE/lcoes.csv"

fig, axes = plt.subplots(3,3,figsize=(10,10))
axes_iter = itertools.cycle(axes.flatten())

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax = next(axes_iter)
    df.groupby(df['technology'].str.replace('A|B','')).sum()['annual generation'].plot(kind='pie', ax=ax,
                                                                                       autopct='%1.0f%%', pctdistance=.7, labeldistance=1.2, 
                                                                                       title=f'Annual generation potential for {exp}', ylabel='')

ax = next(axes_iter)
ax.set_visible(False)

# save figure
save_fig(plt.gcf(), "res-annual-generation-potential-share")

In [ ]:
## Share by technology of electricity
# Use LCoEs from the same ESC for all (LCoElectricity does not depend on ESC, only on exporter)
fn = "../resources/networks_supplied/2030_homogeneous/hvdc/{exp}-DE/lcoes.csv"

fig, axes = plt.subplots(3,3,figsize=(10,10))
axes_iter = itertools.cycle(axes.flatten())

for exp, exp_c in sorted(exp_colors.items()):
    df = pd.read_csv(fn.format(exp=exp))
    ax = next(axes_iter)
    df.groupby(df['technology'].str.replace('A|B','')).sum()['capacity'].plot(kind='pie', ax=ax,
                                                                                       autopct='%1.0f%%', pctdistance=.7, labeldistance=1.2, 
                                                                                       title=f'Installable capacity potential for {exp}', ylabel='')

ax = next(axes_iter)
ax.set_visible(False)
# save figure
save_fig(plt.gcf(), "res-potentials")

In [ ]:
## Share by technology of installed capacity (installed in the model ESC HVDC, 2030, 10% homogeneous WACC)
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")#, index_col=[0,1,2,3,4,5,6])
df = df.set_index([c for c in df.columns if c != 'value'], verify_integrity=True)

subcategories = ['Total installed windonshore capacity',
                  'Total installed windoffshore capacity',
                  'Total installed pvplant capacity']

df = df.loc[2030,'homogeneous',:,:,'DE','RES',subcategories]

df = df.reset_index().pivot(index=['esc','exporter'],columns=['subcategory'],values='value')

# Rename to nicer column names / labels
df = df.rename(columns=lambda x: re.match('Total installed (\S+) capacity', x).groups()[0])

df.columns.name=""

fig, axes = plt.subplots(3,3,figsize=(10,10))
axes_iter = itertools.cycle(axes.flatten())

for esc, esc_m in sorted(esc_markers.items()):
    ax = next(axes_iter)
    df.loc[esc].plot(kind='bar', stacked=True, ax=ax, 
                     title=f'{esc}')
    legend = ax.legend()
    legend.set_visible(False)

legend.set_visible(True)
axl = next(axes_iter)
axl.set_visible(False)

fig.suptitle('Installed capacities [MW]')
# save figure
save_fig(plt.gcf(), "res-installed-capacity")

In [ ]:
## Share by technology of generated electricity (in the model ESC HVDC, 2030, 10% homogeneous WACC)
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")#, index_col=[0,1,2,3,4,5,6])
df = df.set_index([c for c in df.columns if c != 'value'], verify_integrity=True)

subcategories = [
    'Total produced electricity from windonshore',
    'Total produced electricity from windoffshore',
    'Total produced electricity from pvplant']

df = df.loc[2030,'homogeneous',:,:,'DE','RES',subcategories]

df = df.reset_index().pivot(index=['esc','exporter'],columns=['subcategory'],values='value')

# Rename to nicer column names / labels
df = df.rename(columns=lambda x: re.match('Total produced electricity from (\S+)', x).groups()[0])

df.columns.name=""

fig, axes = plt.subplots(3,3,figsize=(10,10))
axes_iter = itertools.cycle(axes.flatten())

for esc, esc_m in sorted(esc_markers.items()):
    ax = next(axes_iter)
    df.loc[esc].plot(kind='bar', stacked=True, ax=ax, 
                     title=f'{esc}')
    legend = ax.legend()
    legend.set_visible(False)

legend.set_visible(True)
axl = next(axes_iter)
axl.set_visible(False)
fig.suptitle('Generated electricity [MWh]')
# save figure
save_fig(plt.gcf(), "res-generated-electricity")

In [ ]:
## Costs of energy by ESC

fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")
df = df.set_index(['year','wacc','importer','category','subcategory','esc','exporter'])

df = df.loc[2030, 'homogeneous','DE','general','Cost per MWh delivered']

xs = np.arange((len(esc_markers))) #label locations
xxs = np.arange(len(exp_colors)) #sublocations within each esc
width = .1 #bar widths

fig = plt.figure(figsize=(12,4))
ax = plt.gca()

for (esc,x) in zip(esc_markers.keys(),xs):
    for ((idx, row), xx) in zip(df.loc[esc].sort_values('value').iterrows(), xxs):
        ax.bar(x+xx*width-(xxs.shape[0]-1)*width/2, row['value'], width, color=exp_colors[idx])
        
# Construct legend
legend_exp = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Exporter', marker=None)]
legend_exp += [matplotlib.lines.Line2D([], [], linestyle='None',markersize=10, marker='.',
                                      label=n, color=c) for n,c in exp_colors.items()]

ax.legend(handles=legend_exp)

ax.set_xticks(xs)
ax.set_xticklabels(list(esc_markers.keys()))

ax.set_ylabel('Cost per energy delivered [EUR/MWh]')

# save figure
save_fig(plt.gcf(), "LCoEnergy_per_ESC-EXP")

In [ ]:
## Cost development across years
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")
df = df.set_index(['wacc','importer','category','subcategory','esc','year','exporter'])

df = df.loc['homogeneous','DE','general','Cost per MWh delivered']

years = [2030,2040,2050]

xs = np.arange((len(esc_markers))) #label locations
xxs = np.arange(len(years)) #sublocations within each esc
width = .2 #bar widths

fig = plt.figure(figsize=(10,6))
ax = plt.gca()

for (esc,x) in zip(esc_markers.keys(),xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            ax.scatter(x+xx*width-(xxs.shape[0]-1)*width/2, row['value'], color=exp_colors[idx], marker='.', s=20, zorder=2, ec='k')
        
        ax.plot([x+xx*width-(xxs.shape[0]-1)*width/2]*2, [df.loc[esc,year].min(),df.loc[esc,year].max()], zorder=1, color='lightgray',lw=3)
        ax.text(x+xx*width-(xxs.shape[0]-1)*width/2, 50, year, rotation=90, va='top')
            
ax.set_ylim(0,)
ax.set_xticks(xs)
ax.set_xticklabels(list(esc_markers.keys()))

ax.set_ylabel('Cost per energy delivered [EUR/MWh]')
ax.set_xlabel('ESC')

# save figure
save_fig(plt.gcf(), f'LCoEnergy_per_ESC_{"-".join([str(y) for y in years])}')

In [ ]:
## Cost development across years
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")
df = df.set_index(['wacc','importer','category','subcategory','esc','year','exporter'])

df = df.loc['lowhomogeneous','DE','general','Cost per MWh delivered']

years = [2030,2040,2050]

xs = np.arange((len(esc_markers))) #label locations
xxs = np.arange(len(years)) #sublocations within each esc
width = .2 #bar widths

fig = plt.figure(figsize=(10,6))
ax = plt.gca()

for (esc,x) in zip(esc_markers.keys(),xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            ax.scatter(x+xx*width-(xxs.shape[0]-1)*width/2, row['value'], color=exp_colors[idx], marker='.', s=20, zorder=2, ec='k')
        
        ax.plot([x+xx*width-(xxs.shape[0]-1)*width/2]*2, [df.loc[esc,year].min(),df.loc[esc,year].max()], zorder=1, color='lightgray',lw=3)
        ax.text(x+xx*width-(xxs.shape[0]-1)*width/2, 25, year, rotation=90, va='top')
            
ax.set_ylim(0,)
ax.set_xticks(xs)
ax.set_xticklabels(list(esc_markers.keys()))

ax.set_ylabel('Cost per energy delivered [EUR/MWh]')
ax.set_xlabel('ESC')

# save figure
save_fig(plt.gcf(), f'LCoEnergy_per_ESC_{"-".join([str(y) for y in years])}_lowhomogeneous')

In [ ]:
## Cost development per t H2 across the years
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")
df = df.set_index(['wacc','importer','category','subcategory','esc','year','exporter'])

df = df.loc['homogeneous','DE','general','Cost per MWh delivered']

# Conversion factor to convert MWh (for chemical in each ESC) to 1 t_H2 content
# Derived from LHV/(H2-content by weight in p.u.)
# i.e. MWh_x/t_H2
conversion_factors = {
    'hvdc':          33.3333/1.,
    'pipeline-h2':   33.3333/1.,
    'pipeline-ch4':  (50./3.6)/0.25,
    'shipping-lh2':  33.3333/1.,
    'shipping-lch4': (50./3.6)/0.25,
    'shipping-meoh': (19.3/3.6)/0.125, 
    'shipping-lnh3': (18.6/3.6)/(3/17), 
    'shipping-lohc': 33.3333/1.,
}

# Rescale costs per MWh_th to Costs per t_H2 content
df['value'] = df.apply(lambda x: conversion_factors[x.name[0]]*x['value'], axis=1)/1e3
df.name = 'Cost per kg_H2 delivered [EUR/kg_H2]'

years = [2030,2040,2050]

xs = np.arange((len(esc_markers))) #label locations
xxs = np.arange(len(years)) #sublocations within each esc
width = .2 #bar widths

fig = plt.figure(figsize=(10,6))
ax = plt.gca()

# Plot each vertical year line and each scatter point separately
for (esc,x) in zip(esc_markers.keys(),xs):
    for (year, xx) in zip(years, xxs):
        for idx, row in df.loc[esc, year].iterrows():
            # each scatter/data point. Marked by color of exporter
            ax.scatter(x+xx*width-(xxs.shape[0]-1)*width/2, row['value'], color=exp_colors[idx], marker='.', s=20, zorder=2, ec='k')
        
        # Vertical line per year
        ax.plot([x+xx*width-(xxs.shape[0]-1)*width/2]*2, [df.loc[esc,year].min(),df.loc[esc,year].max()], zorder=1, color='lightgray',lw=3)
        # Year indicator
        ax.text(x+xx*width-(xxs.shape[0]-1)*width/2, 0.1, year, rotation=90, va='bottom')

ax.set_ylim(0,)
ax.set_xticks(xs)
ax.set_xticklabels(list(esc_markers.keys()))

ax.set_ylabel('Cost per hydrogen delivered [EUR/kg-H2]')
ax.set_xlabel('ESC')

# save figure
save_fig(plt.gcf(), f'LCoH_per_ESC_{"-".join([str(y) for y in years])}')

In [ ]:
## Cost composition for selected ESCs, EXPs
fn = "../results/results.csv"
fn = Path(fn)

df = pd.read_csv(fn, sep=";")
df = df.set_index(['year','wacc','importer','category'])

# Used later, assume demand for all ESCs is identical
demand = df.query('subcategory=="Total demand"').iloc[0]['value']

df = df.loc[2030, 'homogeneous','DE','cost'].reset_index(drop=True)
selected_exps = sorted(['DE','AR','ES'])
selected_escs = sorted(['hvdc','pipeline-h2','shipping-lohc','shipping-meoh'])

df = df[(df['esc'].isin(selected_escs)) & df['exporter'].isin(selected_exps)]
df = df.pivot(index=['esc','exporter'],columns='subcategory',values='value')

# Aggregate columns and nicer names
# Order here also directs the order in the plot later
columns_to_aggregate = {
    'PV/wind':'windonshore|windoffshore|pvplant',
    'water desalination/storage':'(seawater desalination|clean water tank storage)',
    'battery storage':'battery',
    'electrolysis':'electrolysis',
    'storage H2':'hydrogen storage tank|demand buffer',
    'heat pump':'industrial heat pump medium temperature',
    'DAC':'direct air capture',
    'storage CO2':'CO2 (liquefaction|storage tank)',
    'chemical LOHC': 'LOHC chemical',
    'synthesis MeOH': 'methanolisation',
    'storage MeOH':'General liquid hydrocarbon storage \(product\)',
    'storage LOHC':'LOHC (unloaded|loaded) DBT storage',
    'HVDC': 'HVDC',
    'pipeline H2 (g)':'H2 \(g\) pipeline( compressor)?',
    'shipping LOHC':'LOHC transport ship',
    'shipping MeOH':'MeOH transport ship',
    '(de-) hydrogenation LOHC': 'LOHC (de)?hydrogenation',
}

# Rename and aggregate
for c_new,c in columns_to_aggregate.items():
    tmp = df.filter(regex=c, axis=1)
    if tmp.empty:
        continue
    df = df.drop(tmp.columns, axis=1)
    df[c_new] = tmp.sum(axis=1)

number_of_colors = 10
cmap = plt.get_cmap('tab10', number_of_colors)
colors = itertools.cycle([cmap(i/number_of_colors) for i in np.arange(number_of_colors)])

fig = plt.figure(figsize=(10,6))
ax = plt.gca()

tdf = df.copy()

legend_handles = []

for v in list(columns_to_aggregate.keys())[::-1]: # backwards iteration necessary
    
    if any([v.startswith(s) for s in ['shipping LOHC', 'pipeline H2', 'HVDC',
                                      'storage MeOH', 'chemical LOHC',
                                     ]]):
        # Do not use a new color to reduce the color variety
        # the meaning of the color is inferable from the ESC
        # The list to skip is determined by the backwards order of the dict 'columns_to_aggregate'
        print(f"skipping new color for {v}")
    else:
        # New color for new technology
        c = next(colors)
    
    # Plotting a stacked grouped bar plot is stupid
    # we plot the bars and then plot over them with the lower values
    # need to sum() manually and create dedicated dataframes
    (tdf.sum(axis=1).to_frame(name='sum').reset_index().pivot(index='esc',columns='exporter', values='sum')/demand).plot(kind='bar',
                                                                                                                ax=ax,
                                                                                                                color=[c] #pandas needs - for whatever reason - an iterable containing a tuple for RGBA (instead of just 'c')
                                                                                                               )
    tdf = tdf.drop(columns=v)
    
    # Add appropriate legend entry for technology
    legend_handles += [matplotlib.lines.Line2D([], [], linestyle='None', markersize=10, markeredgecolor='black',  marker='s',
                                               label=v, color=c)]

# Header for legend
legend_handles = [matplotlib.lines.Line2D([], [], color='None', linestyle='None', markersize=0, label='Technology', marker=None)] + legend_handles
ax.legend(handles=legend_handles, bbox_to_anchor=(1.05, 1), loc='upper left')

# axis labels
ax.set_ylabel('Cost per MWh chemical delivered [EUR/MWh]')
ax.set_xlabel('ESC and exporting country')
plt.xticks(rotation=0) # Remove automatic rotation from pandas

# Add labels for exporters
width = ax.patches[0].get_width()
for x in ax.get_xticks():
    for i, exp in enumerate(selected_exps):
        ax.text(x + width*(i-1.25), 1, exp)

# save figure
save_fig(plt.gcf(), f'cost-compositions_selected-ESC-EXP')